In [ ]:
# Cell 1: Imports & helper displays

import cv2
import numpy as np
import matplotlib.pyplot as plt

def imshow(img, title=None, size=(12,8)):
    """Display BGR image using matplotlib (converted to RGB)."""
    if img is None:
        print("Image is None")
        return
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=size)
    plt.imshow(img_rgb)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()


In [ ]:
# Cell 2: Keypoint detection, description, and matching

def create_feature_detector():
    """
    Prefer SIFT (opencv-contrib). Fallback to ORB if SIFT not available.
    """
    if hasattr(cv2, "SIFT_create"):
        return cv2.SIFT_create()
    try:
        return cv2.xfeatures2d.SIFT_create()
    except:
        return cv2.ORB_create(5000)


def detect_and_compute(img, detector):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, desc = detector.detectAndCompute(gray, None)
    return kp, desc


def match_descriptors(desc1, desc2, use_l2=True, ratio=0.75):
    """
    BFMatcher + Lowe ratio filter.
    """
    if desc1 is None or desc2 is None:
        return []

    if use_l2:
        bf = cv2.BFMatcher(cv2.NORM_L2)
    else:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)

    matches = bf.knnMatch(desc1, desc2, k=2)

    good = []
    for m, n in matches:
        if m.distance < ratio * n.distance:
            good.append(m)

    return good


def estimate_homography(kp1, kp2, matches, thresh=5.0):
    if len(matches) < 4:
        return None, None

    src = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    dst = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)

    H, mask = cv2.findHomography(src, dst, cv2.RANSAC, thresh)
    return H, mask


In [ ]:
# Cell 3: Warping using homography + canvas creation

def warp_two_images(img1, img2, H):
    """
    Warp img2 to img1 coordinate space → return large canvas + translation.
    """
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # Corners of img2
    corners2 = np.float32([[0,0],[w2,0],[w2,h2],[0,h2]]).reshape(-1,1,2)
    warped_corners2 = cv2.perspectiveTransform(corners2, H)

    # Corners of img1
    corners1 = np.float32([[0,0],[w1,0],[w1,h1],[0,h1]]).reshape(-1,1,2)

    all_corners = np.concatenate((corners1, warped_corners2), axis=0)

    [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 5)
    [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 5)

    tx, ty = -xmin, -ymin
    Ht = np.array([[1,0,tx],[0,1,ty],[0,0,1]])

    size = (xmax - xmin, ymax - ymin)

    pano = cv2.warpPerspective(img2, Ht @ H, size)
    pano[ty:ty+h1, tx:tx+w1] = img1

    return pano, (tx, ty), Ht


In [ ]:
# Cell 4: Full Laplacian pyramid based multi-band blending

def gaussian_pyramid(img, levels):
    gp = [img.astype(np.float32)]
    for _ in range(levels):
        img = cv2.pyrDown(img)
        gp.append(img.astype(np.float32))
    return gp


def laplacian_pyramid(gp):
    lp = []
    for i in range(len(gp)-1):
        up = cv2.pyrUp(gp[i+1])
        if up.shape[:2] != gp[i].shape[:2]:
            up = cv2.resize(up, (gp[i].shape[1], gp[i].shape[0]))
        lp.append(gp[i] - up)
    lp.append(gp[-1])
    return lp


def reconstruct_from_laplacian(lp):
    img = lp[-1]
    for i in range(len(lp)-2, -1, -1):
        img = cv2.pyrUp(img)
        if img.shape[:2] != lp[i].shape[:2]:
            img = cv2.resize(img, (lp[i].shape[1], lp[i].shape[0]))
        img = img + lp[i]
    return img


def multiband_blend(img1, img2, mask, levels=6):
    """
    mask: 0 = img2, 1 = img1
    """
    # Gaussian pyramids
    gp1 = gaussian_pyramid(img1, levels)
    gp2 = gaussian_pyramid(img2, levels)
    gpm = gaussian_pyramid(mask.astype(np.float32), levels)

    # Laplacian pyramids
    lp1 = laplacian_pyramid(gp1)
    lp2 = laplacian_pyramid(gp2)

    blended_pyr = []

    # blend each level
    for L1, L2, M in zip(lp1, lp2, gpm[::-1]):
        if M.ndim == 2:
            M = np.repeat(M[:, :, None], 3, axis=2)
        if M.shape[:2] != L1.shape[:2]:
            M = cv2.resize(M, (L1.shape[1], L1.shape[0]))
        blended = L1 * M + L2 * (1 - M)
        blended_pyr.append(blended)

    result = reconstruct_from_laplacian(blended_pyr)
    return np.clip(result, 0, 255).astype(np.uint8)


In [ ]:
# Cell 5: Masks + final panorama stitching

def create_masks(pano, t, img1, img2, H, Ht):
    h, w = pano.shape[:2]
    tx, ty = t

    # img1 mask
    mask1 = np.zeros((h, w), np.float32)
    h1, w1 = img1.shape[:2]
    mask1[ty:ty+h1, tx:tx+w1] = 1.0

    # img2 mask
    mask2 = (cv2.warpPerspective(img2, Ht @ H, (w, h)).sum(axis=2) > 0).astype(np.float32)

    # Overlap mask
    overlap = (mask1 > 0) & (mask2 > 0)

    mask = mask1.copy()
    if overlap.any():
        ys, xs = np.where(overlap)
        xmin, xmax = xs.min(), xs.max()
        width = max(1, xmax - xmin)

        ramp = np.clip((xmax - np.arange(w)) / width, 0, 1)

        for y in range(h):
            if overlap[y].any():
                mask[y] = ramp

    # Smooth mask
    mask = cv2.GaussianBlur(mask, (41,41), 15)
    mask = (mask - mask.min()) / (mask.max() - mask.min() + 1e-6)

    return mask


def stitch(img1, img2, levels=6):
    det = create_feature_detector()
    kp1, d1 = detect_and_compute(img1, det)
    kp2, d2 = detect_and_compute(img2, det)

    use_l2 = hasattr(cv2, "SIFT_create")
    matches = match_descriptors(d1, d2, use_l2)

    print("Matches:", len(matches))

    H, _ = estimate_homography(kp1, kp2, matches)
    if H is None:
        raise Exception("Homography failed")

    pano, t, Ht = warp_two_images(img1, img2, H)

    # Prepare blending inputs
    img1_on = np.zeros_like(pano)
    tx, ty = t
    h1, w1 = img1.shape[:2]
    img1_on[ty:ty+h1, tx:tx+w1] = img1

    img2_on = cv2.warpPerspective(img2, Ht @ H, (pano.shape[1], pano.shape[0]))

    mask = create_masks(pano, t, img1, img2, H, Ht)

    result = multiband_blend(img1_on, img2_on, mask, levels)

    return result


In [ ]:
# Cell 6: Example usage — replace with your own images

imgA = cv2.imread("left.jpg")     # <-- fill path
imgB = cv2.imread("right.jpg")    # <-- fill path

if imgA is None or imgB is None:
    print("Replace the file paths with actual images.")
else:
    pano = stitch(imgA, imgB)
    imshow(pano, "Final Panorama")
    cv2.imwrite("panorama_output.jpg", pano)
    print("Saved panorama_output.jpg")
